In [1]:
# Check Phrasal paper to see how they split the corpus
# they only use en-fr from autodesk, not en-de

In [2]:
import codecs
from collections import Counter, OrderedDict, defaultdict

In [3]:
# train_source_file = '/home/chris/projects/constrained_decoding/proto/autodesk_constrained_decoding_corpus/autodesk.train.en.bpe'
# train_target_file = '/home/chris/projects/constrained_decoding/proto/autodesk_constrained_decoding_corpus/autodesk.train.de.bpe'

source_tm_file = '/media/1tb_drive/parallel_data/wmt16_IT_domain/in_domain_train/indomain_training/indomain.de-en.en.bpe'
target_tm_file = '/media/1tb_drive/parallel_data/wmt16_IT_domain/in_domain_train/indomain_training/indomain.de-en.de.bpe'


In [4]:
# LOAD dev source, get all constraints, write json file with those constraints

In [5]:
def get_segments_from_file(filename):
    with codecs.open(filename, encoding='utf8') as inp:
        for i, l in enumerate(inp):
            yield l.strip()

In [6]:
source_seg_iter = get_segments_from_file(source_tm_file)
target_seg_iter = get_segments_from_file(target_tm_file)

In [7]:
term_pair_map = {k:v for k,v in zip(source_seg_iter, target_seg_iter)
                 if len(k) > 10 and len(k) < 50
                 and len(v) > 10 and len(v) < 50
                 and abs(len(k) - len(v)) < 15}
src_rules = [src for src in term_pair_map.keys()]

In [8]:
# create terminology spotter
from semantic_annotator.spotting import MatchSpotter

term_spotter = MatchSpotter(rules=src_rules)

In [9]:
# import the tokenized and bpe encoded dev lines
# prepped_dev_lines = codecs.open('/home/chris/projects/constrained_decoding/proto/autodesk_constrained_decoding_corpus/autodesk.dev.1000.en.bpe',
#                                  encoding='utf8').read().strip().split('\n')

# prepped_dev_lines = codecs.open('/home/chris/projects/constrained_decoding/proto/autodesk_constrained_decoding_corpus/en-fr/autodesk.dev.1000.en.bpe',
#                                  encoding='utf8').read().strip().split('\n')

prepped_dev_lines = codecs.open('/media/1tb_drive/parallel_data/wmt16_IT_domain/in_domain_test/refs/en-de/Batch3a_en.bpe',
                                 encoding='utf8').read().strip().split('\n')

In [10]:
for k,v in term_pair_map.items()[:100]:
    print(u'key: {} val: {}'.format(k,v))

key: " Ju@@ iced ( series ) " val: " Ju@@ iced ( Spiel@@ es@@ erie ) "
key: Pat@@ tern color : val: Muster@@ farbe :
key: _ Top of Page ( Header ) val: O _ ben ( Seiten@@ kopf )
key: " Az@@ tec Code " val: Az@@ tec@@ -Code
key: NOR@@ MD@@ IST val: NOR@@ M@@ VER@@ T
key: Mind@@ View val: Mind@@ View
key: 13@@ x@@ 19 val: 13@@ x@@ 19
key: < = : Less than or equal to val: < = : kleiner gleich
key: Not With Report Header val: Nicht mit Bericht@@ sk@@ opf
key: The < quote > Module < / quote > val: Die < quote > Module < / quote >
key: app@@ end : value is not an object val: app@@ end : Der Wert ist kein Objekt
key: of the sum of val: der Summe von
key: " Ser@@ endi@@ pity ( software ) " val: " Ser@@ endi@@ pity ( Software ) "
key: No content found in the document val: Das Dokument hat keinen Inhalt .
key: Vis@@ ited Link val: Gesicht@@ eter Link
key: Split main window horizontally val: Haupt@@ fenster wa@@ ager@@ echt aufteilen
key: " RAM drive " val: RA@@ M-@@ Disk
key: Scan directories re

In [11]:
keys_by_len = sorted(src_rules, key=lambda x: len(x))
print(keys_by_len[-100])
print(term_pair_map[keys_by_len[-100]])


" Computer@@ ized maintenance management system "
" Computer@@ ized Maintenance Management System "


In [12]:
dev_term_spots = []
for l in prepped_dev_lines:
    spots = term_spotter.get_spots(l)
    dev_term_spots.append(spots)

dev_term_constraints = []
for text, spots in zip(prepped_dev_lines, dev_term_spots):
    output_constraints = []
    if len(spots) > 0:
        for spot in spots:
            if spot[1] - spot[0] > 1:
                spotted_term = text[spot[0]:spot[1]]
                print(spotted_term)
                mapped_term = term_pair_map[spotted_term]
                output_constraints.append(mapped_term.split())
    dev_term_constraints.append(output_constraints)

Preferences
Screen Sa@@ ver
Control Panel
Preferences
Control Panel
Alternative
Spo@@ t@@ light
Control Panel
Control Panel
Preferences
Preferences
Advanced Options
Alternative
Double click
Select the file
My Computer
Computer Name
My Computer
Computer Name
description
Format@@ ting
lower@@ case
Dis@@ connect
signal strength
Fac@@ e@@ Time
Fac@@ e@@ Time
Fac@@ e@@ Time
Fac@@ e@@ Time
Navig@@ ate
Fac@@ e@@ Time
Fac@@ e@@ Time
Fac@@ e@@ Time
Fac@@ e@@ Time
Fac@@ e@@ Time
Fac@@ e@@ Time
Fac@@ e@@ Time
Fac@@ e@@ Time
Fac@@ e@@ Time
Fac@@ e@@ Time
Fac@@ e@@ Time
Preferences
Account Information
Scroll down
Y@@ C@@ b@@ C@@ r
Purch@@ ased
Choose File
Open System
Control Panel
Choose File
New Account
My Computer
Do Not Di@@ stur@@ b
Do Not Di@@ stur@@ b
Create an account
credit card
notification
notification
Right click
Control Panel
Right click
read-@@ only
Cyber@@ Link
Drag and drop
Blu@@ es@@ creen
configuration
Downlo@@ ading
N@@ vi@@ dia
Back@@ space
small square
Wra@@ p Text
Page Layout
P

In [13]:
len(dev_term_constraints)

1000

In [14]:
import os
import json

# OUTPUT_DIR='/home/chris/projects/constrained_decoding/proto/autodesk_constrained_decoding_corpus'
# OUTPUT_DIR='/home/chris/projects/constrained_decoding/proto/autodesk_constrained_decoding_corpus/en-fr'
OUTPUT_DIR='/media/1tb_drive/parallel_data/wmt16_IT_domain/in_domain_test/refs/en-de/'

with codecs.open(os.path.join(OUTPUT_DIR, 'tm.test.constraints.json'), 'w', encoding='utf8') as out:
    out.write(json.dumps(dev_term_constraints, indent=2))